In [ ]:
#%load_ext autoreload
#%autoreload 2

# Notebook to compile MATLAB executables

- This notebook provides a mechanism to compile the MATLAB executables required by your tool and in compliance with the current CCR MATLAB license.<br />

- The template repository located at https://github.com/GhubGateway/Ghub_Pegasus_WMS_MATLAB_Example provides scripts specific for the ghubex3 tool. You will need to replace the MATLAB scripts in your tool's src directory and the matlabBuild.sh script in your tool's remotebin directory with the scripts required for your tool. You will also need to update the self_binfiles list in this notebook with the MATLAB executables required for your tool.

- The MATLAB executables are compiled on CCR using the MATLAB mcc function.<br />

- For provenance tracking, the compiled MATLAB executables are retained in your tool's bin directory.<br />

- You will only need to compile the MATLAB executables when MATLAB script files are added or updated in your tool's src directory. 


## Steps for using this notebook

1) Update the scripts in the src and remotebin directories with the scripts required for your tool.<br />
2) Update the self_binfiles list in this notebook with the MATLAB executables required for your tool.<br />
3) Save the notebook update.<br />
4) Click the Appmode button.<br />
5) Click the Run Workflow button to compile the MATLAB executables. The MATLAB scripts in your tool's src directory are compiled on CCR and the returned MATLAB executables are moved your tools's bin directory.



In [ ]:
import sys
import os
import getpass
import platform
import shutil
import atexit
import math
import numpy as np
import pandas as pd
import time

import ipywidgets as widgets
from IPython.display import display, HTML, Markdown, clear_output, Image, Javascript
#import xml.etree.ElementTree as et

import hublib
#print (help(hublib))
import hublib.ui as ui
#print (help(ui))
import hublib.use
#print (help(hublib.use))

# Setup paths to executables
scriptpath = os.path.realpath(" ")
        
# Get the parent dirs
self_tooldir = os.path.dirname(scriptpath)

# Setup path to python and bash scripts
self_srcdir = os.path.join(self_tooldir, "src")

# Setup path to python and bash scripts
self_bindir = os.path.join(self_tooldir, "bin")

# Add to PYTHONPATH
sys.path.insert (1, self_bindir)

# Set up path to the current data directory
self_datadir = os.path.join(self_tooldir, "data")

# Set up path to the current doc directory
self_docdir = os.path.join(self_tooldir, "doc")

# Set up path to the current session directory
self_workingdir = os.getcwd()

# Set up path to the user's home directory
self_homedir = os.path.expanduser("~")

# Initialize the dated run directory.
# Workflow results are not available until after a workflow is executed via Pegasus and completes
self_rundir = ""

self_user = getpass.getuser()

# Returned compiled MATLAB executables
self_binfiles = ['deg2utm', 'utm2deg']

# Version of Pegasus
# Note: when switching the version of Pegasus, delete ~/.pegasus/workflow.db
%use pegasus-5.0.1
from buildWrapper import buildWrapper

widget_border_style = '1px solid black'
widget_output_border_style = '1px solid black'

BOLD = '\033[1m'
SUCCESS = '\033[92m'
WARNING = '\033[93m'
FAIL = '\033[91m'
END = '\033[0m'

dropdown_str_width = 16

dropdown_width = '965px'
dropdown_height = '30px'
button_width = '250px'
button_height = '40px'
ui_string_width = '96.5%'
ui_dropdown_width = '96.2%'

#print ('os.access(self_tooldir, os.W_OK): ', os.access(self_bindir, os.W_OK))


In [ ]:
#https://stackoverflow.com/questions/36757301/disable-ipython-notebook-autoscrolling

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}


In [ ]:
#https://api.jquery.com/ready/
HTML('''
<script>
    function scroll_to_top() {
        Jupyter.notebook.scroll_to_top();
    } 
    $( window ).on( "load", scroll_to_top() );
</script>
''')

In [ ]:
# Button styles
HTML('''
<style>.buttontextclass { color:black ; font-size:130%}</style>
''')

In [ ]:
#tooldir = os.path.dirname(os.path.dirname(os.path.realpath(os.path.abspath(__file__))))
#print ('tooldir: ', tooldir)


In [ ]:
 # Run Workflow

maxwalltime = ui.Number(
    name = 'Maximum Walltime',
    description = 'Maximum Walltime [min]',
    units = 'min',
    value = '30.0',
    min = '5.0',
    max = '60.0'
)

workflow_run_options_form = ui.Form([maxwalltime], name = 'Workflow Run Options')

def run_workflow(p):
    
    # print (p) #Button
    #global self_workflow_succeeded
    #self_workflow_succeeded = False
        
    workflow_progress.clear_output()
    #workflow_results.clear_output()
        
    with workflow_progress:
        
        runWorkflowButton.disabled = True
        #show_log_output_button.disabled = True
        
        start_time = time.time()
        
        try:
            
            # All MATLAB scripts in the src directory will be copied to vortex for compiling the MATLAB exeutables.
            # The matlabBuild.sh script in the remotebin directory contains specific instructions for compiling the MATLAB executables
            
            for i in range(len(self_binfiles)):

                binfile = self_binfiles[i]
                binfilepath = os.path.join(self_bindir, binfile)
                if os.path.exists(binfilepath):
                        print ('removing %s.' %binfilepath)
                        os.remove(binfilepath)
                
            #Note: Workflow execution time depends on the current UB CCR workload.
            print ("Pegasus workflow in progress. This will take approximately 10 minutes...")
            
            #'''
            buildWrapper (" ", \
                self_workingdir, self_binfiles, int(maxwalltime.value))
            #'''
            
            print ('\nWorkflow elapsed time: ' + str((time.time() - start_time)/60.0) + ' [min]\n')
            
            # Check if the results files were created and transferred from CCR 
            # to determine if workflow completed successfully
            
            workflow_completed_successfully = True
            for i in range (len(self_binfiles)):
                binfile = self_binfiles[i]
                binfilepath = os.path.join(self_bindir, binfile)
                if os.path.exists(binfile) == True:
                    shutil.move(binfile, binfilepath)
                else:
                    print ('%s not generated by the workflow.' %binfile)
                    workflow_completed_successfully = False

            if workflow_completed_successfully:

                print ('Workflow completed successfully\n')
                        
            else:

                print ('\nWorkflow did not complete successfully')

                filepath = os.path.join(self_workingdir, 'pegasus.analysis')
                if (os.path.exists(filepath)):
                    print ("pegasus.analysis:\n")
                    f = open(filepath, 'r')
                    output = f.read()
                    f.close()
                    print (output)
                    os.remove(filepath)
       
            #finish_workflow_processing()
        
        except Exception as e:
        
            print ('Workflow Exception: %s\n' %str(e))
       
        runWorkflowButton.disabled = False
        #show_log_output_button.disabled = False


# Abort
# Select Kernel Interrupt
#if self_tW.is_alive() == True:
   #self_tW.terminate()

runWorkflowButton = widgets.Button(description="Run Workflow", disabled=False,\
    layout=widgets.Layout(width=button_width, height=button_height),\
    style= {'button_color':'lightgreen','font_weight':'bold'})
runWorkflowButton.add_class("buttontextclass")
runWorkflowButton.on_click (run_workflow)
#help (runWorkflowButton)

# If self_tooldir is a link, running the installed version of the notebook
if os.path.islink(self_tooldir):
    print ('Compiling MATLAB executables is currently disabled for %s -> %s.' %(self_tooldir, os.readlink(self_tooldir)))
    runWorkflowButton.disabled = True         

In [ ]:
display(workflow_run_options_form)
display(runWorkflowButton)

In [ ]:
workflow_progress = widgets.Output(layout={'border': '1px solid black'})
display(workflow_progress)